## LSTM

In [1]:
import pandas as pd
import numpy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
import statistics

In [60]:
df = pd.read_excel('CRA_train_1200.xlsx')
df_test = pd.read_excel('CRA_test_422.xlsx')
#df = df.head(100) # для быстрого теста работы кода
df_test

,Id,pr_txt,Категория,Уровень рейтинга
0,1212,«Эксперт РА» подтвердил рейтинг Трубной Металл...,NaN,NaN
1,1213,«Эксперт РА» подтвердил рейтинг компании «ГК «...,NaN,NaN
2,1214,«Эксперт РА» подтвердил рейтинг компании «Веле...,NaN,NaN
3,1215,АКРА подтвердило кредитный рейтинг Sanymon Cor...,NaN,NaN
4,1216,«Эксперт РА» подтвердил рейтинг компании «НьюТ...,NaN,NaN
...,...,...,...,...
417,1468,«Эксперт-Анализ» понизил прогноз кредитоспосо...,NaN,NaN
418,1472,Рейтинговое агентство «Эксперт» понизило рейт...,NaN,NaN
419,1475,«Эксперт РА» подтвердил рейтинг новой компани...,NaN,NaN
420,1486,"Финансовый анализ строительной компании ""Соз...",NaN,NaN


In [3]:
df['pr_txt'] = df['pr_txt'].str.lower()
df_test['pr_txt'] = df_test['pr_txt'].str.lower()

In [4]:
# перекодируем категории
label_encoder = LabelEncoder()
df['category_encoded'] = label_encoder.fit_transform(df['Уровень рейтинга'])

# разделим данные на обучающие и тестовые
X_train, X_test, y_train, y_test = train_test_split(df['pr_txt'], df['category_encoded'], test_size=0.2, random_state=3)

In [5]:
# конвертируем текстовые данные в tf-idf вектора для будущей обработки моделью
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# посмотрим матрицы
print("X_train_tfidf shape:", X_train_tfidf.shape)
print("X_test_tfidf shape:", X_test_tfidf.shape)

X_train_tfidf shape: (960, 5000)
X_test_tfidf shape: (240, 5000)


In [6]:
# зададим архитектуру модели
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, dropout):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        # Reshape x to (batch_size, sequence_length, input_dim)
        x = x.view(x.size(0), -1, x.size(1))

        h0 = torch.zeros(self.n_layers, x.size(0), self.hidden_dim).to(device)
        c0 = torch.zeros(self.n_layers, x.size(0), self.hidden_dim).to(device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        out = self.softmax(out)
        return out

# зададим гиперпараметры
input_dim = X_train_tfidf.shape[1]
hidden_dim = 128
output_dim = len(df['Уровень рейтинга'].unique())
n_layers = 2
dropout = 0.2

# зададим нагрузку на устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(input_dim, hidden_dim, output_dim, n_layers, dropout).to(device)

# просмотр архитектуры
print(model)

LSTMClassifier(
  (lstm): LSTM(5000, 128, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=128, out_features=17, bias=True)
  (softmax): LogSoftmax(dim=1)
)


In [7]:
# пропишем функцию потерь и оптимизатор
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# конвертируем данные в PyTorch tensors
X_train_tfidf = torch.FloatTensor(X_train_tfidf.toarray()).to(device)
y_train = torch.LongTensor(numpy.array(y_train)).to(device)

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# тренируем модель
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    outputs = model(X_train_tfidf)

    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/1000], Loss: 2.8373
Epoch [2/1000], Loss: 2.8341
Epoch [3/1000], Loss: 2.8308
Epoch [4/1000], Loss: 2.8275
Epoch [5/1000], Loss: 2.8240
Epoch [6/1000], Loss: 2.8203
Epoch [7/1000], Loss: 2.8164
Epoch [8/1000], Loss: 2.8120
Epoch [9/1000], Loss: 2.8074
Epoch [10/1000], Loss: 2.8024
Epoch [11/1000], Loss: 2.7969
Epoch [12/1000], Loss: 2.7904
Epoch [13/1000], Loss: 2.7835
Epoch [14/1000], Loss: 2.7760
Epoch [15/1000], Loss: 2.7673
Epoch [16/1000], Loss: 2.7577
Epoch [17/1000], Loss: 2.7473
Epoch [18/1000], Loss: 2.7358
Epoch [19/1000], Loss: 2.7240
Epoch [20/1000], Loss: 2.7110
Epoch [21/1000], Loss: 2.6973
Epoch [22/1000], Loss: 2.6830
Epoch [23/1000], Loss: 2.6692
Epoch [24/1000], Loss: 2.6552
Epoch [25/1000], Loss: 2.6420
Epoch [26/1000], Loss: 2.6294
Epoch [27/1000], Loss: 2.6199
Epoch [28/1000], Loss: 2.6119
Epoch [29/1000], Loss: 2.6053
Epoch [30/1000], Loss: 2.5991
Epoch [31/1000], Loss: 2.5941
Epoch [32/1000], Loss: 2.5867
Epoch [33/1000], Loss: 2.5799
Epoch [34/1000], Lo

Epoch [269/1000], Loss: 0.0125
Epoch [270/1000], Loss: 0.0124
Epoch [271/1000], Loss: 0.0122
Epoch [272/1000], Loss: 0.0112
Epoch [273/1000], Loss: 0.0132
Epoch [274/1000], Loss: 0.0120
Epoch [275/1000], Loss: 0.0109
Epoch [276/1000], Loss: 0.0113
Epoch [277/1000], Loss: 0.0113
Epoch [278/1000], Loss: 0.0118
Epoch [279/1000], Loss: 0.0107
Epoch [280/1000], Loss: 0.0110
Epoch [281/1000], Loss: 0.0112
Epoch [282/1000], Loss: 0.0102
Epoch [283/1000], Loss: 0.0099
Epoch [284/1000], Loss: 0.0102
Epoch [285/1000], Loss: 0.0104
Epoch [286/1000], Loss: 0.0089
Epoch [287/1000], Loss: 0.0098
Epoch [288/1000], Loss: 0.0097
Epoch [289/1000], Loss: 0.0092
Epoch [290/1000], Loss: 0.0092
Epoch [291/1000], Loss: 0.0092
Epoch [292/1000], Loss: 0.0092
Epoch [293/1000], Loss: 0.0093
Epoch [294/1000], Loss: 0.0097
Epoch [295/1000], Loss: 0.0088
Epoch [296/1000], Loss: 0.0098
Epoch [297/1000], Loss: 0.0083
Epoch [298/1000], Loss: 0.0086
Epoch [299/1000], Loss: 0.0087
Epoch [300/1000], Loss: 0.0086
Epoch [3

Epoch [534/1000], Loss: 0.0023
Epoch [535/1000], Loss: 0.0022
Epoch [536/1000], Loss: 0.0024
Epoch [537/1000], Loss: 0.0021
Epoch [538/1000], Loss: 0.0021
Epoch [539/1000], Loss: 0.0020
Epoch [540/1000], Loss: 0.0020
Epoch [541/1000], Loss: 0.0020
Epoch [542/1000], Loss: 0.0019
Epoch [543/1000], Loss: 0.0020
Epoch [544/1000], Loss: 0.0020
Epoch [545/1000], Loss: 0.0022
Epoch [546/1000], Loss: 0.0022
Epoch [547/1000], Loss: 0.0020
Epoch [548/1000], Loss: 0.0019
Epoch [549/1000], Loss: 0.0020
Epoch [550/1000], Loss: 0.0018
Epoch [551/1000], Loss: 0.0019
Epoch [552/1000], Loss: 0.0019
Epoch [553/1000], Loss: 0.0019
Epoch [554/1000], Loss: 0.0021
Epoch [555/1000], Loss: 0.0019
Epoch [556/1000], Loss: 0.0020
Epoch [557/1000], Loss: 0.0020
Epoch [558/1000], Loss: 0.0020
Epoch [559/1000], Loss: 0.0022
Epoch [560/1000], Loss: 0.0018
Epoch [561/1000], Loss: 0.0019
Epoch [562/1000], Loss: 0.0020
Epoch [563/1000], Loss: 0.0018
Epoch [564/1000], Loss: 0.0019
Epoch [565/1000], Loss: 0.0018
Epoch [5

Epoch [800/1000], Loss: 0.0010
Epoch [801/1000], Loss: 0.0009
Epoch [802/1000], Loss: 0.0009
Epoch [803/1000], Loss: 0.0009
Epoch [804/1000], Loss: 0.0009
Epoch [805/1000], Loss: 0.0009
Epoch [806/1000], Loss: 0.0009
Epoch [807/1000], Loss: 0.0009
Epoch [808/1000], Loss: 0.0009
Epoch [809/1000], Loss: 0.0009
Epoch [810/1000], Loss: 0.0009
Epoch [811/1000], Loss: 0.0008
Epoch [812/1000], Loss: 0.0009
Epoch [813/1000], Loss: 0.0009
Epoch [814/1000], Loss: 0.0009
Epoch [815/1000], Loss: 0.0009
Epoch [816/1000], Loss: 0.0008
Epoch [817/1000], Loss: 0.0009
Epoch [818/1000], Loss: 0.0009
Epoch [819/1000], Loss: 0.0008
Epoch [820/1000], Loss: 0.0009
Epoch [821/1000], Loss: 0.0009
Epoch [822/1000], Loss: 0.0009
Epoch [823/1000], Loss: 0.0009
Epoch [824/1000], Loss: 0.0009
Epoch [825/1000], Loss: 0.0008
Epoch [826/1000], Loss: 0.0009
Epoch [827/1000], Loss: 0.0009
Epoch [828/1000], Loss: 0.0009
Epoch [829/1000], Loss: 0.0009
Epoch [830/1000], Loss: 0.0008
Epoch [831/1000], Loss: 0.0009
Epoch [8

In [8]:
# конвертируем данные в последовательность, а затем в тензор
y_test = torch.tensor(numpy.array(y_test), dtype=torch.long).to(device)
X_test_tfidf_dense = torch.FloatTensor(X_test_tfidf.toarray()).to(device)

model.eval()
with torch.no_grad():
    outputs = model(X_test_tfidf_dense)
    _, predicted = torch.max(outputs, 1)

correct = (predicted == y_test).sum().item()
total = y_test.size(0)
accuracy = correct / total * 100
print(f'Test Accuracy: {accuracy:.2f}%')
# оценка точности модели

Test Accuracy: 65.42%


In [9]:
w_f1_score = f1_score(y_test, predicted, average='weighted')
print(f'F1 Score: {w_f1_score:.4f}')

F1 Score: 0.6509


In [10]:
# сохраним модель
torch.save(model, 'model_LSTM.pth')

In [11]:
# та же модель, но для укрупненных категорий
## LSTM (для укрупненных значений)

df['category_encoded1'] = label_encoder.fit_transform(df['Категория'])

# разделим данные на обучающие и тестовые
X_train1, X_test1, y_train1, y_test1 = train_test_split(df['pr_txt'], df['category_encoded1'], test_size=0.2, random_state=3)

# конвертируем текстовые данные в tf-idf вектора для будущей обработки моделью
X_train_tfidf1 = tfidf_vectorizer.fit_transform(X_train1)
X_test_tfidf1 = tfidf_vectorizer.transform(X_test1)

# посмотрим матрицы
print("X_train_tfidf shape:", X_train_tfidf1.shape)
print("X_test_tfidf shape:", X_test_tfidf1.shape)

# зададим гиперпараметры
input_dim = X_train_tfidf1.shape[1]
hidden_dim = 128
output_dim = len(df['Категория'].unique())
n_layers = 2
dropout = 0.2

model1 = LSTMClassifier(input_dim, hidden_dim, output_dim, n_layers, dropout).to(device)

# просмотр архитектуры
print(model1)

# пропишем функцию потерь и оптимизатор
criterion = nn.NLLLoss()
optimizer = optim.Adam(model1.parameters(), lr=0.001)

# конвертируем данные в PyTorch tensors
X_train_tfidf1 = torch.FloatTensor(X_train_tfidf1.toarray()).to(device)
y_train1 = torch.LongTensor(numpy.array(y_train1)).to(device)

criterion = nn.NLLLoss()
optimizer = optim.Adam(model1.parameters(), lr=0.001)

# тренируем модель
num_epochs = 1000
for epoch in range(num_epochs):
    model1.train()
    optimizer.zero_grad()

    outputs = model1(X_train_tfidf)

    loss = criterion(outputs, y_train1)
    loss.backward()
    optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# конвертируем данные в последовательность, а затем в тензор
y_test1 = torch.tensor(numpy.array(y_test1), dtype=torch.long).to(device)
X_test_tfidf_dense1 = torch.FloatTensor(X_test_tfidf1.toarray()).to(device)

X_train_tfidf shape: (960, 5000)
X_test_tfidf shape: (240, 5000)
LSTMClassifier(
  (lstm): LSTM(5000, 128, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=128, out_features=7, bias=True)
  (softmax): LogSoftmax(dim=1)
)
Epoch [1/1000], Loss: 1.9363
Epoch [2/1000], Loss: 1.9325
Epoch [3/1000], Loss: 1.9288
Epoch [4/1000], Loss: 1.9250
Epoch [5/1000], Loss: 1.9210
Epoch [6/1000], Loss: 1.9168
Epoch [7/1000], Loss: 1.9124
Epoch [8/1000], Loss: 1.9075
Epoch [9/1000], Loss: 1.9023
Epoch [10/1000], Loss: 1.8965
Epoch [11/1000], Loss: 1.8905
Epoch [12/1000], Loss: 1.8834
Epoch [13/1000], Loss: 1.8756
Epoch [14/1000], Loss: 1.8672
Epoch [15/1000], Loss: 1.8577
Epoch [16/1000], Loss: 1.8474
Epoch [17/1000], Loss: 1.8363
Epoch [18/1000], Loss: 1.8235
Epoch [19/1000], Loss: 1.8103
Epoch [20/1000], Loss: 1.7958
Epoch [21/1000], Loss: 1.7811
Epoch [22/1000], Loss: 1.7659
Epoch [23/1000], Loss: 1.7491
Epoch [24/1000], Loss: 1.7342
Epoch [25/1000], Loss: 1.7184
Epoch [26/1000]

Epoch [261/1000], Loss: 0.0048
Epoch [262/1000], Loss: 0.0047
Epoch [263/1000], Loss: 0.0045
Epoch [264/1000], Loss: 0.0047
Epoch [265/1000], Loss: 0.0044
Epoch [266/1000], Loss: 0.0047
Epoch [267/1000], Loss: 0.0045
Epoch [268/1000], Loss: 0.0043
Epoch [269/1000], Loss: 0.0045
Epoch [270/1000], Loss: 0.0046
Epoch [271/1000], Loss: 0.0041
Epoch [272/1000], Loss: 0.0042
Epoch [273/1000], Loss: 0.0042
Epoch [274/1000], Loss: 0.0042
Epoch [275/1000], Loss: 0.0039
Epoch [276/1000], Loss: 0.0042
Epoch [277/1000], Loss: 0.0040
Epoch [278/1000], Loss: 0.0041
Epoch [279/1000], Loss: 0.0041
Epoch [280/1000], Loss: 0.0039
Epoch [281/1000], Loss: 0.0038
Epoch [282/1000], Loss: 0.0037
Epoch [283/1000], Loss: 0.0040
Epoch [284/1000], Loss: 0.0038
Epoch [285/1000], Loss: 0.0039
Epoch [286/1000], Loss: 0.0037
Epoch [287/1000], Loss: 0.0039
Epoch [288/1000], Loss: 0.0034
Epoch [289/1000], Loss: 0.0035
Epoch [290/1000], Loss: 0.0034
Epoch [291/1000], Loss: 0.0035
Epoch [292/1000], Loss: 0.0035
Epoch [2

Epoch [527/1000], Loss: 0.0009
Epoch [528/1000], Loss: 0.0008
Epoch [529/1000], Loss: 0.0008
Epoch [530/1000], Loss: 0.0009
Epoch [531/1000], Loss: 0.0008
Epoch [532/1000], Loss: 0.0009
Epoch [533/1000], Loss: 0.0008
Epoch [534/1000], Loss: 0.0008
Epoch [535/1000], Loss: 0.0009
Epoch [536/1000], Loss: 0.0009
Epoch [537/1000], Loss: 0.0008
Epoch [538/1000], Loss: 0.0008
Epoch [539/1000], Loss: 0.0009
Epoch [540/1000], Loss: 0.0008
Epoch [541/1000], Loss: 0.0009
Epoch [542/1000], Loss: 0.0008
Epoch [543/1000], Loss: 0.0008
Epoch [544/1000], Loss: 0.0008
Epoch [545/1000], Loss: 0.0008
Epoch [546/1000], Loss: 0.0007
Epoch [547/1000], Loss: 0.0008
Epoch [548/1000], Loss: 0.0008
Epoch [549/1000], Loss: 0.0008
Epoch [550/1000], Loss: 0.0009
Epoch [551/1000], Loss: 0.0008
Epoch [552/1000], Loss: 0.0008
Epoch [553/1000], Loss: 0.0008
Epoch [554/1000], Loss: 0.0007
Epoch [555/1000], Loss: 0.0008
Epoch [556/1000], Loss: 0.0008
Epoch [557/1000], Loss: 0.0008
Epoch [558/1000], Loss: 0.0008
Epoch [5

Epoch [793/1000], Loss: 0.0004
Epoch [794/1000], Loss: 0.0004
Epoch [795/1000], Loss: 0.0004
Epoch [796/1000], Loss: 0.0004
Epoch [797/1000], Loss: 0.0004
Epoch [798/1000], Loss: 0.0004
Epoch [799/1000], Loss: 0.0004
Epoch [800/1000], Loss: 0.0004
Epoch [801/1000], Loss: 0.0004
Epoch [802/1000], Loss: 0.0004
Epoch [803/1000], Loss: 0.0004
Epoch [804/1000], Loss: 0.0003
Epoch [805/1000], Loss: 0.0004
Epoch [806/1000], Loss: 0.0004
Epoch [807/1000], Loss: 0.0004
Epoch [808/1000], Loss: 0.0004
Epoch [809/1000], Loss: 0.0004
Epoch [810/1000], Loss: 0.0004
Epoch [811/1000], Loss: 0.0004
Epoch [812/1000], Loss: 0.0004
Epoch [813/1000], Loss: 0.0004
Epoch [814/1000], Loss: 0.0004
Epoch [815/1000], Loss: 0.0004
Epoch [816/1000], Loss: 0.0004
Epoch [817/1000], Loss: 0.0004
Epoch [818/1000], Loss: 0.0004
Epoch [819/1000], Loss: 0.0004
Epoch [820/1000], Loss: 0.0004
Epoch [821/1000], Loss: 0.0004
Epoch [822/1000], Loss: 0.0003
Epoch [823/1000], Loss: 0.0003
Epoch [824/1000], Loss: 0.0004
Epoch [8

In [12]:
model1.eval()
with torch.no_grad():
    outputs = model1(X_test_tfidf_dense1)
    _, predicted1 = torch.max(outputs, 1)

correct1 = (predicted1 == y_test1).sum().item()
total1 = y_test1.size(0)
accuracy1 = correct1 / total1 * 100
print(f'Test Accuracy: {accuracy1:.2f}%')
# оценка точности модели

w_f1_score1 = f1_score(y_test1, predicted1, average='weighted')
print(f'F1 Score: {w_f1_score1:.4f}')

# сохраним модель
torch.save(model1, 'model_LSTM1.pth')

Test Accuracy: 84.17%
F1 Score: 0.8400


In [13]:
print(f'Final Score: {w_f1_score*0.35 + w_f1_score1*0.65:.3f}')

Final Score: 0.774


In [61]:
# загружаем тестовые данные
df_test['pr_txt'] = df_test['pr_txt'].str.lower()

# векторизуем
X_new_tfidf = tfidf_vectorizer.transform(df_test['pr_txt'])
X_new_tfidf = torch.FloatTensor(X_new_tfidf.toarray()).to(device)

model.eval()
with torch.no_grad():
    outputs = model(X_new_tfidf)
    _, predicted = torch.max(outputs, 1)

# запишем значения в столбец
df_test['Уровень рейтинга'] = predicted
df_test

,Id,pr_txt,Категория,Уровень рейтинга
0,1212,«эксперт ра» подтвердил рейтинг трубной металл...,NaN,1
1,1213,«эксперт ра» подтвердил рейтинг компании «гк «...,NaN,2
2,1214,«эксперт ра» подтвердил рейтинг компании «веле...,NaN,13
3,1215,акра подтвердило кредитный рейтинг sanymon cor...,NaN,2
4,1216,«эксперт ра» подтвердил рейтинг компании «ньют...,NaN,2
...,...,...,...,...
417,1468,«эксперт-анализ» понизил прогноз кредитоспосо...,NaN,5
418,1472,рейтинговое агентство «эксперт» понизило рейт...,NaN,12
419,1475,«эксперт ра» подтвердил рейтинг новой компани...,NaN,0
420,1486,"финансовый анализ строительной компании ""соз...",NaN,6


In [57]:
df_test['Уровень рейтинга'] = pd.to_numeric(df_test['Уровень рейтинга'], errors='coerce')
df_test

,Id,pr_txt,Категория,Уровень рейтинга
0,1212,«эксперт ра» подтвердил рейтинг трубной металл...,None,1
1,1213,«эксперт ра» подтвердил рейтинг компании «гк «...,None,2
2,1214,«эксперт ра» подтвердил рейтинг компании «веле...,None,13
3,1215,акра подтвердило кредитный рейтинг sanymon cor...,None,2
4,1216,«эксперт ра» подтвердил рейтинг компании «ньют...,None,2
...,...,...,...,...
417,1468,«эксперт-анализ» понизил прогноз кредитоспосо...,None,5
418,1472,рейтинговое агентство «эксперт» понизило рейт...,None,12
419,1475,«эксперт ра» подтвердил рейтинг новой компани...,None,0
420,1486,"финансовый анализ строительной компании ""соз...",None,6


In [65]:
mapping = {0: 'C',
           1: 'B-',
           2: 'B',
           3: 'B+',
           4: 'BB-',
           5: 'BB',
           6: 'BB+',
           7: 'BBB-',
           8: 'BBB',
           9: 'BBB+',
           10: 'A-',
           11: 'A',
           12: 'A+',
           13: 'AA-',
           14: 'AA',
           15: 'AA+',
           16: 'AAA'
           }

df_test['Уровень рейтинга'] = df_test['Уровень рейтинга'].replace(mapping)

In [62]:
model1.eval()
with torch.no_grad():
    outputs = model1(X_new_tfidf)
    _, predicted = torch.max(outputs, 1)

# запишем значения в столбец
df_test['Категория'] = predicted
df_test

,Id,pr_txt,Категория,Уровень рейтинга
0,1212,«эксперт ра» подтвердил рейтинг трубной металл...,0,1
1,1213,«эксперт ра» подтвердил рейтинг компании «гк «...,0,2
2,1214,«эксперт ра» подтвердил рейтинг компании «веле...,5,13
3,1215,акра подтвердило кредитный рейтинг sanymon cor...,0,2
4,1216,«эксперт ра» подтвердил рейтинг компании «ньют...,0,2
...,...,...,...,...
417,1468,«эксперт-анализ» понизил прогноз кредитоспосо...,2,5
418,1472,рейтинговое агентство «эксперт» понизило рейт...,4,12
419,1475,«эксперт ра» подтвердил рейтинг новой компани...,0,0
420,1486,"финансовый анализ строительной компании ""соз...",0,6


In [64]:
df_test['Категория'] = df['Категория'].replace(mapping)
df_test

,Id,pr_txt,Категория,Уровень рейтинга
0,1212,«эксперт ра» подтвердил рейтинг трубной металл...,A,1
1,1213,«эксперт ра» подтвердил рейтинг компании «гк «...,BB,2
2,1214,«эксперт ра» подтвердил рейтинг компании «веле...,A,13
3,1215,акра подтвердило кредитный рейтинг sanymon cor...,AAA,2
4,1216,«эксперт ра» подтвердил рейтинг компании «ньют...,BBB,2
...,...,...,...,...
417,1468,«эксперт-анализ» понизил прогноз кредитоспосо...,B,5
418,1472,рейтинговое агентство «эксперт» понизило рейт...,BB,12
419,1475,«эксперт ра» подтвердил рейтинг новой компани...,A,0
420,1486,"финансовый анализ строительной компании ""соз...",A,6


In [66]:
df_test.to_excel('df_test.xlsx', index=False)